In [28]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

t_cost = pd.read_csv('salesman2.csv', index_col=0)
N = len(t_cost.columns) # number of cities

# Create an empty model
m = gp.Model('tsm')

# Create variables for which route to select
route = m.addVars(range(1,N+1), range(1,N+1), vtype=GRB.BINARY, name="route")

# Create helper variables to eliminate subtours
u = m.addVars(range(1,N+1), name="u")

# All routes from node to itself are 0
m.addConstrs(route[i,i] == 0 for i in range(1,N+1))

# enter each city once constraints
m.addConstrs(sum(route[i,j] for j in range(1,N+1)) == 1 for i in range(1,N+1))

# exit each city once constraints
m.addConstrs(sum(route[i,j] for i in range(1,N+1)) == 1 for j in range(1,N+1))

# subtour elimination constraints
for i in range(1,N+1):
    for j in range(2,N+1):
        m.addConstr(u[i] + route[i,j] <= u[j] + (N+1 - 1) * (1 - route[i,j]))
# fixes that the first order of node to visit is 1
m.addConstr(u[1]==0)
    
# nonnegativity constraints for u
m.addConstrs(u[i] >= 0 for i in range(1,N+1))

# Compute optimal solution
obj = sum( t_cost.iloc[i-1,j-1]*route[i,j] for i in range(1,N+1) for j in range(1,N+1) )
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

# Print optimal value of the objective function
print('\nValue of objective function: %g' % m.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m.getVars():
    if v.x>0.5:
        print('%s = %g' % (v.varName, v.x))

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 341 rows, 306 columns and 1397 nonzeros
Model fingerprint: 0x25474225
Variable types: 17 continuous, 289 integer (289 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 67 rows and 18 columns
Presolve time: 0.01s
Presolved: 274 rows, 288 columns, 1264 nonzeros
Variable types: 16 continuous, 272 integer (272 binary)

Root relaxation: objective 2.000000e+00, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.00000    0   21          -    2.00000      -     -    0s
     0     0   18.23810    0   29          -   18.23810      -     -    0s
H    0     0            